In [1]:
import pandas as pd
import requests
import time
import pandasql as psql
from etl_resources import sqlite_connection, get_api_key, get_symbol_list

In [ ]:
def build_prices_dataset():
    
    '''
    This function uses the alphavantage api to pull pricing information for 
    S&P 100 equities on a weekly basis. Weekly data has a much longer history.
    The dataframes are transposed and written to the local sqlite database for
    future use.
    '''
    
    api_key = get_api_key()
    symbol_list = get_symbol_list()
    con = sqlite_connection()
    
    for symbol in symbol_list:
        
        time.sleep(10) # Handle the rate limiter
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={api_key}"
        
        try:
            
            print(f"Parsing {symbol}")
            response = requests.get(url)
            json = response.json()['Weekly Time Series']

            df = pd.DataFrame.from_dict(json, orient='index')
            df.columns = [col.split(' ')[1] for col in df.columns]
            df = df.reset_index()
            df.rename(columns = {'index': 'date'}, inplace = True)
            df['ticker'] = symbol

            df.to_sql(name='weekly_prices', if_exists='append', index=False, con=con)
        
        except:
            # TODO: Better failure logging
            print(f"Failed: {url}")


In [ ]:
def process_price_datatypes():
    
    '''
    This function cleans up the datatypes for the weekly prices table.
    '''
    
    con = sqlite_connection()
        
    print("Processing price datatypes")
    df = pd.read_sql("select * from weekly_prices",con=con)
    df['date'] = pd.to_datetime(df['date'])
    
    nums = ['open','high','low','close','volume']
    for field in nums:
        df[field] = pd.to_numeric(df[field])
        
    df.to_sql("weekly_prices_clean",if_exists='replace',con=con)

In [ ]:
def process_quaterly_prices():
    
    con = sqlite_connection()
    cur = con.cursor()
    print("Working on weekly_prices_qtr")
    
    try:
        cur.execute("drop table weekly_prices_qtr")
    except:
        print("weekly_prices_qtr doesnt exist")


    cur.execute(f"select distinct ticker from weekly_prices_clean")
    res = cur.fetchall()
    res = [val[0] for val in res]
    
    for ticker in res:
        qry = f'''
    SELECT
       wc.date,
       wc.close,
       x.ticker,
       x.year,
       x.quarter 
    FROM
       weekly_prices_clean wc 
       INNER JOIN
          (
             SELECT
                MAX(w.date) AS last_date,
                c.quarter,
                c.year,
                w.ticker 
             FROM
                weekly_prices_clean w 
                LEFT JOIN
                   calendar c 
                   ON c.date = w.date 
             WHERE
                c.quarter IS NOT NULL and w.ticker='{ticker}'
             GROUP BY
                w.ticker,
                c.quarter,
                c.year 
             ORDER BY
                w.ticker,
                c.year,
                c.quarter 
          )
          x 
          ON x.last_date = wc.date 
          AND x.ticker = wc.ticker 
    ORDER BY
       x.ticker,
       x.year,
       x.quarter

        '''

        df = pd.read_sql(qry, con=con)
        df = df.set_index(['date', 'quarter','year', 'ticker'])
        perc_df = df.pct_change()
        diff_df = df.fillna(0)
        diff_df = diff_df.diff()
        combined_df = pd.merge(perc_df, diff_df, on=['date','quarter','year','ticker'])
        combined_df.columns = [col.replace('_x','_pct').replace('_y','_val') for col in combined_df.columns]
        final_df = pd.merge(df,combined_df, on=['date','quarter','year','ticker'])

        final_df.to_sql(name='weekly_prices_qtr',if_exists='append', con=con)
    

In [ ]:
build_prices_dataset()
process_price_datatypes()
#process_quaterly_prices()